In [1]:
from utils import load_embedding, parse_tup, cos_sim, cos_dist
from sklearn.decomposition import PCA
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()

/h/224/cameron/miniconda3/envs/pyspark_env/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
TIME_FRAME = "monthly"
embedding = load_embedding("/h/224/cameron/Political-Subreddit-Embedding/trained_embeddings/vecs_0.0028_15.0.txt",split=False)
embedding

,1,2,3,4,5,6,7,8,9,10,...,141,142,143,144,145,146,147,148,149,150
subreddit,,,,,,,,,,,,,,,,,,,,,
SquaredCircle,0.134779,0.043821,-0.011876,-0.066840,0.020694,-0.055040,0.017334,0.004694,-0.082625,0.124690,...,-0.061495,-0.183028,-0.099551,0.031485,-0.040188,0.143292,0.040536,-0.008464,0.041863,-0.172806
AskReddit,0.135559,0.008985,-0.037187,0.000455,0.022365,-0.006085,-0.017524,0.040978,-0.054059,0.069235,...,-0.039066,-0.043172,-0.106157,0.105634,-0.045787,0.063289,0.057239,-0.198356,0.061649,-0.023330
politics,-0.042422,0.024062,-0.165820,0.046046,-0.080393,0.024717,0.044797,0.094564,-0.090970,0.013700,...,0.034073,-0.087685,-0.129094,0.110916,0.047369,0.014214,0.025798,-0.121198,0.155587,-0.057969
memes,0.002449,-0.014680,-0.031889,-0.051864,0.044783,-0.055175,0.063965,0.009760,-0.135631,0.106458,...,0.024190,-0.042757,-0.033878,0.041545,-0.001212,0.076116,0.036539,-0.102034,0.054330,-0.073219
teenagers,0.026058,0.069684,0.024581,0.009624,-0.029134,0.076384,0.078542,0.052331,-0.106552,0.096511,...,0.013686,-0.014345,-0.004146,0.055941,-0.009106,0.085962,0.030374,-0.028468,0.069988,-0.039540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NRV,0.122154,0.130032,-0.243602,-0.032611,-0.038153,-0.014279,-0.065262,-0.001528,-0.099809,-0.018441,...,-0.054521,-0.062958,-0.112747,0.098669,-0.049754,-0.015509,0.117170,-0.216936,0.008118,-0.123790
UTT,0.135902,0.057751,-0.200740,-0.035059,-0.005987,-0.000219,-0.083139,0.033362,-0.119045,0.125617,...,-0.019219,-0.092823,-0.096780,0.095344,-0.114785,0.023995,0.075017,-0.151421,0.048709,-0.065192
MMU,0.119556,0.049138,-0.134668,0.009505,0.023105,-0.025217,-0.026836,-0.026815,-0.198236,0.021817,...,-0.075906,-0.067976,-0.035603,-0.009426,-0.084691,0.068536,-0.040373,-0.134642,0.068508,-0.132696


# Analogies

In [3]:
def analogy(sub_a,sub_ap,sub_b,embedding,return_score=False):
    """
        sub_a is to sub_ap, as sub_b is to ??
        sub_ap + sub_b - sub_a
        returns the subreddit that best answers the analogy
    """
    assert sub_a in embedding.index.values, "{} is not in the dataset".format(sub_a)
    assert sub_ap in embedding.index.values, "{} is not in the dataset".format(sub_ap)
    assert sub_b in embedding.index.values, "{} is not in the dataset".format(sub_b)
    vec_a = embedding.loc[sub_a].to_numpy().flatten()
    vec_ap = embedding.loc[sub_ap].to_numpy().flatten()
    vec_b = embedding.loc[sub_b].to_numpy().flatten()
    vec_bp = vec_ap + (vec_b - vec_a)
    all_other_vecs = embedding[~embedding.index.isin([sub_a,sub_ap,sub_b])]
    similarity = np.apply_along_axis(lambda x : cos_sim(x,vec_bp), 1, all_other_vecs)
    winner_ind = similarity.argmax()
    if return_score:
        return all_other_vecs.index.values[winner_ind], similarity[winner_ind]
    return all_other_vecs.index.values[winner_ind]


# a = "BaemyKlobaechar"
# ap = "ElizabethWarren"
# b = "JoeBiden"
# print("r/{} is to r/{} as r/{} is to r/{}".format(a,ap,b,analogy(a,ap,b,embedding)))
# a = "JoeBiden"
# ap = "SandersForPresident"
# b = "BaemyKlobaechar"
# print("r/{} is to r/{} as r/{} is to r/{}".format(a,ap,b,analogy(a,ap,b,embedding)))

# a = "socialism"
# ap = "Libertarian"
# b = "EnoughCommieSpam"
# print("r/{} is to r/{} as r/{} is to r/{}".format(a,ap,b,analogy(a,ap,b,embedding)))
# # a = "SandersForPresident"
# ap = "ElizabethWarren"
# b = "BaemyKlobaechar"
# print("r/{} is to r/{} as r/{} is to r/{}".format(a,ap,b,analogy(a,ap,b,embedding)))

In [4]:
tqdm.pandas()

def get_possible_fits(a,ap,embedding,k=20):
    """
        Returns possible b and bp's for an analogy starting with a and ap
        Iterate through each vector in the embedding, have that act as the b and find bp.
        Sort through ones that match the best and return top k.
    """
    # Find subreddits that are relatively close to the base "a" subreddit, and take closest 100
    pos_fits = embedding.copy().apply(lambda x : cos_sim(np.array(x),embedding.loc[a]),axis=1,result_type="expand").sort_values(0,ascending=False).head(50)    
    pos_fits = pos_fits.reset_index().set_index("subreddit")
    print(pos_fits)
    # Search for best possible fits in ENTIRE embedding for the analogy a is to ap as [subreddits closest to a] is bp
    pos_fits[["sub_to","sim"]] = pos_fits.progress_apply(lambda x : analogy(a,ap,x.name,embedding,True),axis=1,result_type="expand")
    pos_fits = pos_fits.reset_index().rename({"subreddit":"sub_from"},axis=1)[["sub_from","sub_to","sim"]]
    # Bring everything together
    pos_fits = pos_fits.sort_values("sim",ascending=False)
    pos_fits = pos_fits[~(pos_fits["sub_from"].isin([a,ap]) | pos_fits["sub_to"].isin([a,ap]))]
    return  pos_fits.head(k)

# a = "GunsAreCool"
# ap = "progun"
# # p1 = get_pos_fits(a,ap,embedding,k=20)
# # print(p1)
ap = "exchristian"
a = "Christianity"
# a = "AskMen"
# ap = "AskWomen"
# a = "CallOfDuty"
# ap = "modernwarfare"
p2 = get_possible_fits(a,ap,embedding)
print("{} is to {} best analogy endings".format(a,ap))
print(p2)

/h/224/cameron/miniconda3/envs/pyspark_env/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


                             0
subreddit                     
Christianity          1.000000
TrueChristian         0.725204
Christian             0.697255
Reformed              0.696752
Christians            0.694194
DebateAChristian      0.680342
theology              0.672094
Bible                 0.670041
NoFapChristians       0.662602
AskAChristian         0.634147
Creation              0.627549
Christianmarriage     0.626053
DebateReligion        0.605598
Catholicism           0.598479
PrayerRequests        0.595367
Anglicanism           0.579806
AcademicBiblical      0.575425
OpenChristian         0.571348
GayChristians         0.562075
DebateAnAtheist       0.560972
TraditionalCatholics  0.560298
prolife               0.552354
religion              0.545078
CatholicMemes         0.543147
DebateEvolution       0.535672
OrthodoxChristianity  0.533206
CatholicDating        0.533001
RadicalChristianity   0.530219
agnostic              0.518377
Abortiondebate        0.508717
TrueAthe


Christianity is to exchristian best analogy endings
                sub_from                sub_to       sim
5       DebateAChristian       DebateAnAtheist  0.712000
12        DebateReligion       DebateAnAtheist  0.702676
19       DebateAnAtheist           TrueAtheism  0.681900
30           TrueAtheism       DebateAnAtheist  0.657739
29        Abortiondebate             prochoice  0.657387
24       DebateEvolution       DebateAnAtheist  0.656718
46            espn2watch      Raptors76ersLive  0.654451
45             prochoice              abortion  0.640478
43        tucker_carlson           TheNewRight  0.636113
9          AskAChristian       DebateAnAtheist  0.635254
1          TrueChristian            excatholic  0.633214
20  TraditionalCatholics            excatholic  0.633019
15           Anglicanism            excatholic  0.632223
40         conservatives  conservativecartoons  0.624736
6               theology      AcademicBiblical  0.623867
38                progun      liber

In [5]:
from itertools import combinations

def predict_analogies(analogy_names,embedding,frac=0.5,show_prog=True):
    fp = "/h/224/cameron/Political-Subreddit-Embedding/external_datasets/validation/{}.csv"
    data_frames = [pd.read_csv(fp.format(name)).sample(frac=frac) for name in analogy_names]
    pairs = []
    for analogy_df in data_frames:
        tuples = [tuple(x) for x in analogy_df.to_numpy()]
        analogy_pairs = list(combinations(tuples,2))
        pairs += analogy_pairs
    pred = []
    actual = []
    tups = []
    p = tqdm(pairs) if show_prog else pairs
    print(len(p))
    for (a,ap),(b,bp) in p:
        try: 
            p = analogy(a.strip(),ap.strip(),b.strip(),embedding)
            pred.append(p)
            actual.append(bp)
            tups.append(((a,ap),(b,bp)))
        except:
            pass
    return np.array(pred),np.array(actual), tups

predict_analogies(["football","basketball","baseball","hockey","populous_cities"],None,frac=1,show_prog=False)

def score(pred,actual):
    return (pred == actual).mean() * 100, (pred == actual).sum()

data_sets = ["football","basketball","baseball","hockey","populous_cities"]



pred, actual, tuples = predict_analogies(data_sets,embedding,frac=0.1)
accuracy, num_correct = score(pred,actual)
print("Out of {} analogies, {} were predicted correctly ({}%)".format(len(pred),num_correct,accuracy))
for p,act,((a,ap),(b,_)) in zip(pred,actual,tuples):
    if not p==act: print("{} is to {} as {} is to {} (predicted as: {})".format(a,ap,b,act,p))


2462


18

Out of 18 analogies, 15 were predicted correctly (83.33333333333334%)
nyc is to NYGiants as nashville is to Tennesseetitans (predicted as: ockytop)
toronto is to torontoraptors as tulsa is to Thunder (predicted as: oklahoma)
Charlotte is to NorthCarolina as chicago is to illinois (predicted as: chicagofood)


## Evaluate temporal embedding

In [6]:

embedding  = load_embedding("/h/224/cameron/Political-Subreddit-Embedding/trained_embeddings/temporal/{}/{}_vecs_0.0028_15.txt".format(TIME_FRAME,TIME_FRAME),split=False)

#Parse out date and subreddit from single index
t = embedding.index.map(lambda x: parse_tup(x,date_str="%d/%m/%Y"))
t = list(map(tuple,t))
embedding.index = pd.MultiIndex.from_tuples(t)
embedding = embedding.rename_axis(("subreddit", "date")).reorder_levels(['date','subreddit']).sort_values(["date","subreddit"])
embedding

1         2         3         4    \
date       subreddit                                                     
2019-01-01 1000ccplus           0.027411  0.007928 -0.109678  0.070665   
           100thieves           0.047010  0.089473  0.001833 -0.059464   
           1022                 0.030958  0.029023  0.031724 -0.102030   
           1200isfineIGUESSugh  0.093791 -0.000252  0.002537  0.099571   
           1200isjerky          0.154488  0.057273 -0.018089  0.143122   
...                                  ...       ...       ...       ...   
2019-12-01 zoemains             0.031772  0.026607  0.012095  0.093933   
           zoloft              -0.015821  0.053895 -0.015671  0.113680   
           zombies              0.013930 -0.102829 -0.008978  0.036583   
           zootopia            -0.071352  0.014392  0.082478  0.164587   
           zurich               0.085357 -0.071645 -0.176382 -0.059816   

                                     5         6         7         8    \
date       subreddit                                                     
2019-01-01 1000ccplus          -0.069335  0.036830  0.001487  0.000331   
           100thieves           0.099490  0.032410 -0.004810 -0.101613   
           1022                 0.064858 -0.189088  0.021475 -0.152345   
           1200isfineIGUESSugh  0.105243  0.032816  0.049385 -0.133215   
           1200isjerky          0.095987  0.111545  0.023787 -0.171960   
...                                  ...       ...       ...       ...   
2019-12-01 zoemains             0.100424 -0.005248 -0.067083 -0.065163   
           zoloft               0.057699  0.082132  0.021603  0.030836   
           zombies             -0.085338 -0.022046 -0.069338  0.005291   
           zootopia             0.077086 -0.052253 -0.072058 -0.082849   
           zurich               0.092763  0.107670 -0.131393 -0.077398   

                                     9         10   ...       141       142  \
date       subreddit                                ...                       
2019-01-01 1000ccplus          -0.048816 -0.112004  ...  0.026002 -0.014206   
           100thieves           0.066657  0.035370  ...  0.001671 -0.028480   
           1022                 0.027263 -0.012738  ... -0.068612  0.034416   
           1200isfineIGUESSugh  0.117549 -0.070538  ...  0.016156 -0.073431   
           1200isjerky          0.020003 -0.040333  ...  0.080083 -0.089391   
...                                  ...       ...  ...       ...       ...   
2019-12-01 zoemains             0.068851  0.039087  ...  0.113103  0.030495   
           zoloft               0.009351 -0.128631  ... -0.082551 -0.100545   
           zombies             -0.076649  0.006726  ...  0.009584  0.051413   
           zootopia             0.004199 -0.033957  ...  0.102761 -0.031334   
           zurich               0.005380  0.068437  ...  0.140103  0.114566   

                                     143       144       145       146  \
date       subreddit                                                     
2019-01-01 1000ccplus          -0.148223  0.058003 -0.006749 -0.219159   
           100thieves           0.000751  0.120835  0.051996 -0.121277   
           1022                -0.161073  0.011924  0.000407 -0.135506   
           1200isfineIGUESSugh -0.073697  0.045124 -0.045725 -0.030220   
           1200isjerky         -0.109960  0.059116 -0.077729 -0.000051   
...                                  ...       ...       ...       ...   
2019-12-01 zoemains             0.039067  0.074273 -0.039663 -0.104319   
           zoloft              -0.115522  0.002291  0.068781 -0.151290   
           zombies              0.003283  0.016159 -0.068776 -0.000747   
           zootopia            -0.038930  0.126400 -0.038243  0.063623   
           zurich              -0.123826  0.036651 -0.008637 -0.080500   

                                     147       148       149       150  
date       subreddit                           

In [13]:
data_sets = ["football","basketball","baseball","hockey","populous_cities"]
dates = []
scores = []
for date, sub_embedding in tqdm(embedding.groupby(level=0)):
    # Drop date from multiindex -> single index
    sub_embedding = sub_embedding.droplevel(level=0)
    pred, actual, _ = predict_analogies(data_sets,sub_embedding,frac=0.5,show_prog=False)
    acc, _ = score(pred,actual)
    scores.append(acc)
    dates.append(date) 
scores = np.array(scores)/100
print(dates,"\n///\n",scores)   


[Timestamp('2019-01-01 00:00:00'), Timestamp('2019-02-01 00:00:00'), Timestamp('2019-03-01 00:00:00'), Timestamp('2019-04-01 00:00:00'), Timestamp('2019-05-01 00:00:00'), Timestamp('2019-06-01 00:00:00'), Timestamp('2019-07-01 00:00:00'), Timestamp('2019-08-01 00:00:00'), Timestamp('2019-09-01 00:00:00'), Timestamp('2019-10-01 00:00:00'), Timestamp('2019-11-01 00:00:00'), Timestamp('2019-12-01 00:00:00')] 
///
 [0.68556701 0.57044674 0.66151203 0.68556701 0.67182131 0.66151203
 0.72164948 0.66151203 0.66315789 0.63448276 0.75945017 0.68900344]


In [14]:
import plotly.express as px
args = {
    "line_shape": "spline",
    "title":  "Temporal Community Embedding Analogy Accuracy",
    "template": "simple_white",
    "labels": {"x": "Date", "y":"Analogy Accuracy"}
}

u = np.mean(scores)
d = np.std(scores)
layout = {
            "yaxis": {"tickformat": '%',"range":[0,1] },
            # Mean Horizontal
            "shapes": [{
                'type': 'line',
                'x0': dates[0],
                'y0': u,
                'x1': dates[-1],
                'y1': u,
                'line': {
                    'color': 'rgb(50, 171, 96)',
                    'width': 2},
                },
            ],
            "annotations": [{"x":"2019-06-24","y":u,"text":"u={:.2f}%, σ={:0.2f}".format(u*100,d)}],
            "font": {"size": 25}
         }

fig = px.line(x=dates,y=scores,**args)
fig.update_layout(layout)
# fig.write_html("visualizations/temporal/{}_accuracy.html".format(TIME_FRAME))
fig.show()